# Week 4: Gradient Descent

#### Design Thinking and Predictive Analytics for Data Products

Earlier in this course, we learned how to to use linear regression in Python. Here we will do two examples of [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent) using regression. The first example will feel familiar to what we have done previously, while the second one will show us how to do the same task using TensorFlow. if you dont know what Tensorflow is yet, feel free to read more [here](https://www.tensorflow.org/about).

## Part 1: Gradient Descent in Python

### The Data

We will be looking at the `household_power_consumption.csv` file. This dataset contains roughly 2 million sample ratings of electric power consumption in one household. We are going to use this data to predict the `Kitchen power comsumption` as a function of the `Global power consumption`.

Link: https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption

### Reading the Data

Specify the path of the file. You may need to change the given path according to your local environment. This should be familiar to you by now, if not Homework 1 from this couse should be a good guide for you.

In [ ]:
path = "./datasets/household_power_consumption.txt"
f = open(path, 'r')

# Note how the data is separated here
dataset = []
header = f.readline().strip().split(';')
for line in f:
    line = line.split(';')
    dataset.append(line)

header

Note: `Sub_metering_1` = `Kitchen power`

In [ ]:
# Note: Sub_metering_1 = Kitchen power
# You can read about what each column stands for by clicking on the download link for the data above

# Which columns hold Global power and Kitchen power?
print(header.index('Global_active_power'), header.index('Sub_metering_1')) 

In [ ]:
# Notice these are all strings and not numerical values!
# How might we fix this?
dataset[2]

In [ ]:
dataset = [d for d in dataset if d != 'NA'] # Make sure we are only using data points with values

shortData = dataset[:100000] # Feel free to use a larger value here. For this example's purposes, this value will do.
print(len(shortData), len(dataset))

### Solving our Data Format Problem

In the next cell we are using two functions of Python you may not be familiar with: `map()` along with `try`/`except`. If you would like to read more about them, here are their respective links ([map](https://docs.python.org/3/library/functions.html#map)) and ([try/except](https://docs.python.org/3/tutorial/errors.html)). 

For now you don't have to worry about how they work, but I encourage you to play around with the for-loop and figure out what happens without the `try/except` portion. (Hint: there's a reason we have to use it here.) As for `map()`, here we are simply using it to fix the problem we had above, by getting float values out of our strings.

In [ ]:
def strToFloat(data):
    data = list(map(float, data))
    
    
for i in range(1,len(shortData)):
    try:
        strToFloat(shortData[i][2:]) # [2:] so that we only grab the numerical values, not the Time and Date
    except ValueError:
        shortData[i] = 'NA' # What might we be doing here?
        
shortData = [d for d in shortData if d != 'NA'] # This counteracts the excpet part from the for loop above

### Keeping It Simple

For our demonstration we are only using one feature vector, `Global Power`. If you would like to predict `Kitchen Power` (or any other value) as a function of more than one feature, you would add those extra features here. How might you go about this?

In [ ]:
def feature(datum): # This is where the function comes from!
    feat = [1, float(datum[2])] # Global Power
    return feat

In [ ]:
X = [feature(d) for d in shortData] # Feature vector (Global Power)
y = [float(d[6]) for d in shortData] # Kitchen Power
K = len(X[0])
theta = [0.0]*K
theta[0] = sum(y) / len(y)

Below are a few utility functions to compute the inner product and norm of a vector. These are also implemented in various libraries (e.g. numpy), and using the library functions would likely result in a more efficient implementation, but the functions are written here just for simplicity in our example.

In [ ]:
#Defining the inner product of a vector
def inner(x,y):
    return sum([a*b for (a,b) in zip(x,y)])

#Defining the 2-norm of a vector
def norm(x):
    return sum([a*a for a in x]) # equivalently, inner(x,x)

### The Derivative Function

Most important is our function to compute the derivative. The expression being computed here is the derivative of the objective (the Mean Squared Error) with respect to the parameters (`theta`), at our current estimate of `theta`.

In [ ]:
def derivative(X, y, theta):
    dtheta = [0.0]*len(theta) # Initialize the derivative vector to be a vector of zeros
    K = len(theta)
    N = len(X)
    MSE = 0 # Compute the MSE as we go, just to print it for debugging
    for i in range(N):
        error = inner(X[i],theta) - y[i]
        for k in range(K):
            dtheta[k] += 2*X[i][k]*error/N # See lectures to understand how this expression was derived
        MSE += error*error/N
    return dtheta, MSE

Next let's choose a Learning Rate. For this test's sake, let's call it .01. The smaller you set this, the longer your function will take to converge below, though the added computations will give a more accurate `theta` value.

In [ ]:
learningRate = .01

Now we iteratively call our derivative function to improve our estimate of `theta`, by following the direction of the derivative. The details of this function are quite difficult to get right, e.g. if the learning rate or the convergence criteria are not set well, the function may not produce a good solution.

In [ ]:
while (True):
    dtheta,MSE = derivative(X, y, theta)
    m = norm(dtheta)
    print("norm(dtheta) = " + str(m) + " MSE = " + str(MSE))
    for k in range(K):
        theta[k] -= learningRate * dtheta[k]
    if m < 0.01: break

In [ ]:
theta

$$Power_K = theta_0 + Power_G*theta_1$$

Now we have a function that determines the `Kitchen Power` as a function of the `Global Power`! Although this is acceptable it was both complicated and slow (computationally). Libraries like TensorFlow can help alleviate these issues and give the same results.

## Part 2: Gradient Descent in Tensorflow

Most of the operations will be similar so we won't explain every single function like we did above, but know that the logic is very similar.

In [ ]:
import tensorflow as tf #Start by importing TensorFlow

path = "./datasets/household_power_consumption.txt"
f = open(path, 'r')

#Read the data the same as above
dataset = []
header = f.readline().strip().split(';')
for line in f:
    line = line.split(';')
    dataset.append(line)
    
header # Verify this is the same header as above

Remember to grab however many values you want to run this on and run our code to weed out any non-numeric values.

In [ ]:
dataset = [d for d in dataset if d != 'NA'] #Make sure we are only using data points with values

shortData = dataset[:100000] # Feel free to use a larger value here. For this example's purposes, this value will do

def strToFloat(data):
    data = list(map(float, data))
    
    
for i in range(1,len(shortData)):
    try:
        strToFloat(shortData[i][2:]) # [2:] so that we only grab the numerical values, not the Time and Date
    except ValueError:
        shortData[i] = 'NA'
        
shortData = [d for d in shortData if d != 'NA'] # This counteracts the except part from the for-loop above

### Trying Something New

This time, let's add a few more features to our feature vector. Let's try calculating our `Kitchen Power` as a function of `Global Power` and `Laundry energy`. How will this change the `theta` vector we are given at the end? 

Note: `Sub_metering_2` = `Laundry Power`

In [ ]:
def feature(datum):
    feat = [1, float(datum[2]), float(datum[7])] # Global Active Power, Laundry Power
    return feat

X = [feature(d) for d in dataset] #Getting our feature vector as defined above
y = [float(d[6]) for d in dataset] #Again grabbing the Kitchen power

In [ ]:
y = tf.constant(y, shape=[len(y),1])
K = len(XTf[0]) #same as in other example

If you would like to read what `tf.constant()` does in general, [here's](https://www.tensorflow.org/api_docs/python/tf/constant) the link. In this instance it is creating a 1-D tensor populated with the values of the list `y`.

The main advantage of TensorFlow is that we don't have to compute the gradient - TensorFlow will compute it for us. Instead, we have to implement our _objective_ (i.e., the MSE) in terms of TensorFlow operations:

In [ ]:
def MSE(X, y, theta):
  return tf.reduce_mean((tf.matmul(X,theta) - y)**2)

Next we tell TensorFlow that `theta` is our vector of variables to be optimized (we also specify its shape and initial values):

In [ ]:
theta = tf.Variable(tf.constant([0.0]*K, shape=[K,1]))

Here we select an optimizer, which is a learning rate.

In [ ]:
optimizer = tf.train.AdamOptimizer(0.01)

Then we tell TensorFlow that our MSE function is the objective to be optimized.

In [ ]:
objective = MSE(X,y,theta)

Then we tell TensorFlow that this objective should be minimized (i.e., we are trying to minimize an error, rather than maximizing an accuracy), and initialize the session.

In [ ]:
train = optimizer.minimize(objective)
init = tf.global_variables_initializer()

Finally we run 1000 iterations of gradient descent. Note how fast this is compared to our own implementation!

In [ ]:
for iteration in range(1000):
  cvalues = sess.run([train, objective])
  print("objective = " + str(cvalues[1]))

Once gradient descent has converged, we can print out the results (i.e., `theta`):

In [ ]:
with sess.as_default():
  print(MSE(X, y, theta).eval())
  print(theta.eval())

Now we have a new function to determine the `Kitchen Power` consumption! It looks something like this:

$$Power_K = theta_0 + (Power_G*theta_1) + (Power_L*theta_2)$$

Go ahead and either 
    1. Change the original feature function to be a function of these two variables, OR
    2. Change your tensorflow feature function to only be a function of Global Power
    
Compare the two results. The similarity between the two should serve as an indication of the power of libraries like TensorFlow and how they can aid us in solving problems like this.

## You're all done!

You are now able to use gradient descent in two different ways! We encourage you to try this out on other datasets that interest you and see what kind of interesting data you can come up with.